<a href="https://colab.research.google.com/github/PeerChristensen/NLP-Demos/blob/main/SpaCy_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A quick introduction to SpaCy


### Getting started



Upgrading SpaCy is necessary for use with Colab. 
Here, we download a small Danish model. 

Other options include `da_core_news_lg` and a transfomer model - `da_core_news_trf` (i.e. Maltehb/danish-bert-botxo - also available through Huggingface)

More information about Danish models [here](https://spacy.io/models/da)

In [2]:
!pip install --upgrade spacy
!python -m spacy download da_core_news_sm

import spacy
nlp = spacy.load("da_core_news_sm")

     |████████████████████████████████| 6.0 MB 18.0 MB/s 
     |████████████████████████████████| 628 kB 62.3 MB/s 
     |████████████████████████████████| 181 kB 59.8 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 10.1 MB 25.4 MB/s 
     |████████████████████████████████| 451 kB 72.4 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 19.1 

In [3]:
text = "Mens pandemien er i vækst over store dele af Europa, er en ny og bekymrende coronavariant ved navn Omikron opdaget i Sydafrika."

doc = nlp(text)

### What's in the `doc` container?

The `doc` variable contains a a sequence of tokens with a bunch of information attached.

Here, we create and use a function to return a dataframe with some key info.

In [6]:
import pandas as pd

def df_from_text(text):

  cols = ("token", "lemma", "POS", "POSexplained", "stopword")
  rows = []

  for word in doc:
    row = [word.text, word.lemma_, word.pos_, spacy.explain(word.pos_), word.is_stop]
    rows.append(row)

  return pd.DataFrame(rows, columns=cols)

In [7]:
df_from_text(text)

,token,lemma,POS,POSexplained,stopword
0,Mens,Mens,SCONJ,subordinating conjunction,True
1,pandemien,pandemien,NOUN,noun,False
2,er,være,VERB,verb,True
3,i,i,ADP,adposition,True
4,vækst,vækst,NOUN,noun,False
5,over,over,ADP,adposition,True
6,store,stor,ADJ,adjective,False
7,dele,del,NOUN,noun,False
8,af,af,ADP,adposition,True
9,Europa,Europa,PROPN,proper noun,False


### Extracting named entities

Most models can extract named **persons**, **locations** and **organizations**

In [ ]:
text2 = "Du forbinder nok jul med pebernødder, risalamande og brunkager, men hvordan smager julen i Bulgarien, Italien og Sverige?"

def get_entities(text):

  cols = ("entity", "label", "start", "end")
  rows = []

  doc = nlp(text)
  for ent in doc.ents:
    row = [ent.text, ent.label_, ent.start_char, ent.end_char]
    rows.append(row)

  return pd.DataFrame(rows, columns=cols)
    
get_entities(text2)

With the Displacy library, visualizing named entities within text is easy.

In [ ]:
from spacy import displacy

text3 = "Det kommer næppe som en stor overraskelse, at Donald Trump endnu engang ikke har noget pænt at sige om hertuginde Meghan, der er gift med Storbritanniens prins Harry.  Den forhenværende amerikanske præsident har tidligere udtalt, at han »ikke er fan af Meghan«, og nu forklarer Donald Trump sig i et interview til den tidligere UKIP-formand, Nigel Farage.  I interviewet, der sendes i aften på britisk tv, beskylder Donald Trump hertuginde Meghan for at »mangle respekt« og for at »skade« dronning Elizabeth, skriver The Daily Mail. Donald Trump kommenterer også beskyldninger om, at hertuginden skulle være manipulerende over for sin mand. Hun skulle ifølge anklagerne være den direkte årsag til,  at hertugparret i begyndelsen af 2020 valgte at træde ud af den kongelige familie for i stedet at flytte USA."

doc = nlp(text3)

displacy.render(doc, style="ent", jupyter=True)

### Tokenization


For downstream tasks such as text classification or topic modelling, we might try different ways of preprocessing texts.

In this case, we create a function that only outputs lemmatized nouns transformed to lowercase.

In [ ]:
def noun_lemmatizer(sentence):
    """Using SpaCy to lemmatize and extract nouns"""
    tokens = nlp(sentence)
    tokens = [word.lemma_.lower() for word in tokens if word.pos_ == "NOUN"]
    return tokens

In [ ]:
text4 = "Du forbinder nok jul med pebernødder, risalamande og brunkager, men hvordan smager julen i Bulgarien, Italien og Sverige?"

noun_lemmatizer(text4)

### Getting noun phrases

In [ ]:
text5 = "Jeg er den stolte ejer af to store røde biler."

doc = nlp(text5)

for chunk in doc.noun_chunks:
    print(f"> {chunk.text}")

### Leaner pipelines

The default processing pipeline includes a tagger, a lemmatizer, a parser and an entity recognizer. Each pipeline component returns the processed Doc, which is then passed on to the next component.

We may choose to only keep some of the processing components in the pipeline.
This can help you process text faster.

In this case, we disable several unnecessary steps for NER.



In [ ]:
texts = [text, text2, text3, text4, text5]

for doc in nlp.pipe(texts, disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"]):

    print([(ent.text, ent.label_) for ent in doc.ents])